# Calculating DCJ distance between two unsigned, circular and/or linear genomes with no duplicates

Credit:
- https://github.com/mlliou112/py-dcj (python version)
- ChatGPT for converting to Julia 

In [1]:
# import statements
using Base.Iterators
using Parameters
using DataStructures

abstract type AbstractGene end

In [2]:
# telomere

@with_kw struct Telomere <: AbstractGene
    reverse::Bool=false  # TODO
end

function Base.isequal(a::Telomere, b)
    return typeof(a) == typeof(b)
end

function Base.show(t::Telomere)
    return "."
end

In [3]:
# gene

@with_kw struct Gene <: AbstractGene
    id::Int
    dna::Char
    reverse::Bool=false
end

function Base.isequal(a::Gene, b::Gene)
    return typeof(a) == typeof(b) && a.id == b.id
end

function Base.show(m::Gene)
    if m.reverse
        print("-" * m.dna) # * string(m.id))
    else
        print(m.dna) # string(m.id))
    end
end

In [4]:
# chromosome

mutable struct Chromosome
    genes::Vector{AbstractGene}
end

function Chromosome(genes::String, id_counter::Ref{Int64}, id_to_char::Dict{Int, Char}, char_to_id::Dict{Char, Int64}, target::Bool)
    content = AbstractGene[]

    if !((genes[1] == '.') == (genes[end] == '.'))
        throw(ArgumentError("Linear geneosome must start and end with telomeres."))
    end

    for (i, gene) in pairs(genes)
        dna = only(lowercase(string(gene)))
        rev = isuppercase(gene)
        telomere = gene == '.'        
        if telomere 
            push!(content, Telomere())
        else  # gene 
            id = id_counter[]

            if target  
                id_to_char[id] = gene
                if gene in keys(char_to_id)
                    push!(char_to_id[dna], id)
                else 
                    char_to_id[dna] = id
                end 
                id_counter[] += 1

            else  # source str 
                ## assuming no duplicates
                id = char_to_id[dna]
            end
            push!(content, Gene(id, dna, rev))
        end 
    end    
    
    return Chromosome(content)
end

Chromosome

In [5]:
# Genome

mutable struct Genome
    data::Vector{Chromosome}
    id_to_char::Dict{Int, Char}
    char_to_id::Dict{Char, Int}
end

In [6]:
# funcs convert bt genome & str 
function genome_to_string(A)
end


function string_to_genome(s, id_counter, id_to_char, char_to_id, target)
     """
    Converts string to type Genome 
    """

    chrom_list_str = Vector{String}()

    chrom = ""
    new_chrom = true 
    linear = false

    for (i, g) in pairs(s)
        # if creating a new chromosome 
        if new_chrom 
            if chrom == ""
                linear = g == '.'
            else  # chrom isn't empty 
                linear = chrom[1] == '.'
            end 

            chrom = chrom * g
            new_chrom = false  
            if i == length(s) # last element is circular chrom 
                push!(chrom_list_str, chrom)
               
            end 
        
        # appending more genes to chromosome 
        else  
            if linear  
                chrom = chrom * g 
                if g == '.' || i == length(s)  # find another telomere or end of str
                    push!(chrom_list_str, chrom) 
                    chrom = ""
                    new_chrom = true 
                end 
            else  # circular
                if g =='.'  # close circular chrom (won't incude telomere in chrom)
                    push!(chrom_list_str, chrom)
                    chrom = "."  # put telomere in next chrom 
                    new_chrom = true 
                elseif i == length(s)  # reach end of str 
                    chrom = chrom * g
                    push!(chrom_list_str, chrom)
                    chrom = "."
                    new_chrom = true 
                else 
                    chrom = chrom * g
                end 
            end 
        end 
    end

    # convert chrom_list_str (list of strings) to chrom_list
    chrom_list = Chromosome[]
    
    for c_str in chrom_list_str
        c = Chromosome(c_str, id_counter, id_to_char, char_to_id, target)
        push!(chrom_list, c)
    end 
    
    return Genome(chrom_list, id_to_char, char_to_id)
end 

string_to_genome (generic function with 1 method)

In [7]:
# gene end (used in adj)

@with_kw struct GeneEnd
    gene::AbstractGene
    head::Bool=true
end

# adjacency
mutable struct Adjacency 
    left::GeneEnd
    right::GeneEnd
end

function Base.isequal(a::GeneEnd, b::GeneEnd)
    return a.id == b.id && a.head == b.head
end

function Base.show(ge::GeneEnd)
    if ge.gene == Telomere() 
        print(".")
        return 
    end 
    
    if ge.head
        print(string(ge.gene.dna) *":h")
    else
        print(string(ge.gene.dna) * ":t")
    end

end

function Base.isequal(a::Adjacency, b::Adjacency)
     # order of the gene ends doesn't matter
    if a.left == b.left && a.right == b.right || a.left == b.right && a.right == b.left  
        return true 
    end 
    return false 
end 

function Base.show(adj::Adjacency)
    left = ""
    right = ""
    
    if adj.left.gene == Telomere() 
        left = "."
    else 
        left_gene = adj.left
        (left_gene.head == true) ? left = left_gene.gene.dna * ":h" : left = left_gene.gene.dna * ":t" 
    end
    
    if adj.right.gene == Telomere() 
        right = "."
    else 
        right_gene = adj.right
        (right_gene.head == true) ? right = right_gene.gene.dna * ":h" : right = right_gene.gene.dna * ":t"
    end 

    print("(", left, ",", right, ")")
end 

function Base.show(adj::Adjacency, add_spacer::Bool)
    left = ""
    right = ""
    
    if adj.left.gene == Telomere() 
        left = "."
    else 
        left_gene = adj.left
        (left_gene.head == true) ? left = left_gene.gene.dna * ":h" : left = left_gene.gene.dna * ":t" 
    end
    
    if adj.right.gene == Telomere() 
        right = "."
    else 
        right_gene = adj.right
        (right_gene.head == true) ? right = right_gene.gene.dna * ":h" : right = right_gene.gene.dna * ":t"
    end 

    if add_spacer 
        print("(", left, "---", right, ")")
    else
        print("(", left, ",", right, ")")
    end 
end 


function Base.show(adj_list::Vector{Adjacency})
    for adj in adj_list 
        show(adj) 
    end 
end 

function Base.show(adj_list::Vector{Adjacency}, idx1::Int, idx2::Int)
    r = min(idx1, idx2)
    s = max(idx1, idx2)
    for (i, adj) in pairs(adj_list)  
        show(adj, i == r || i == s) 
    end 
end 

function Base.show(adj_set::Set{Adjacency})
    for adj in adj_set 
        show(adj) 
    end 
end 

In [8]:
# genome to adj list and set 

function genome_to_adj_listset_helper(chrom::Chromosome, genes::Vector{AbstractGene}, adj_list::Vector{Adjacency}, adj_set::Set{Adjacency})
    linear = genes[1] == Telomere()
    
    # handle first gene end 
    if linear 
        left_end_adj = GeneEnd(Telomere(), false)
    else  # circular 
        left_end_adj = GeneEnd(genes[1], !genes[1].reverse)
    end 

    # handle gene bt first and last 
    for gene in genes
        if gene == Telomere() || gene == genes[1] 
            continue 
        end 
        reversed = gene.reverse  
        
        right_gene_adj = GeneEnd(gene, reversed)  # if reversed, right_end_adj is head (& left_end_adj is tail) 
        adj = Adjacency(left_end_adj, right_gene_adj)

        push!(adj_list, adj)
        push!(adj_set, adj)

        left_end_adj = GeneEnd(gene, !reversed)
    end 

    # handle last gene end 
    if linear 
        right_end_adj = GeneEnd(Telomere(), false)
    else   
        right_end_adj = GeneEnd(genes[1], genes[1].reverse)
    end

    return left_end_adj, right_end_adj

end 


function genome_to_adj_listset(genome::Genome)
    adj_list = Vector{Adjacency}()
    adj_set = Set{Adjacency}()

    for chrom in genome.data
        genes = chrom.genes
        reversed = genes[1].reverse  
        
        if length(genes) == 1  # single gene edge case   
            left_end_adj = GeneEnd(genes[1], reversed)  # if reversed, left_end_adj is head (& right_end_adj is tail) 
            right_end_adj = GeneEnd(genes[1], !reversed) 
        elseif length(genes) == 2  # two gene edge case 
            gene1 = genes[1]
            gene2 = genes[2]

            left_end_adj = GeneEnd(gene1, !gene1.reverse)
            right_end_adj = GeneEnd(gene2, gene2.reverse) 
            adj = Adjacency(left_end_adj, right_end_adj)
            push!(adj_list, adj)
            push!(adj_set, adj)

            reversed = genes[2].reverse 
            left_end_adj = GeneEnd(gene2, !gene2.reverse)
            right_end_adj = GeneEnd(gene1, gene1.reverse) 
        else # > 2 gene in chrom  
            left_end_adj, right_end_adj = genome_to_adj_listset_helper(chrom, genes, adj_list, adj_set)
        end 
        
        adj = Adjacency(left_end_adj, right_end_adj)

        push!(adj_list, adj)
        push!(adj_set, adj)
    end 
    
    return adj_list, adj_set
end 

genome_to_adj_listset (generic function with 1 method)